This notebook contains an example for teaching.

# The Effect of Gun Ownership on Gun-Homicide Rates - proceeding

In this lab, we estimate the effect of gun ownership on the homicide rate by a neural network.

In [7]:
using Pkg
Pkg.add("CSV"), using CSV
Pkg.add("DataFrames"), using DataFrames
Pkg.add("StatsModels"), using StatsModels
Pkg.add("GLM"), using GLM
Pkg.add("Random"), using Random
using Statistics, Plots, FixedEffectModels, MLDataUtils, MLBase

    Updating registry at `C:\Users\PC\.julia\registries\General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Manifest.toml`


First, we need to load and preprocess the data.

In [32]:
data = CSV.File("../data/gun_clean.csv") |> DataFrame;
println("Number of rows: ",size(data,1))
println("Number of columns: ",size(data,2))

Number of rows: 3900
Number of columns: 415


In [33]:
################################# Create Variables ###############################

# Dummy Variables for Year and County Fixed Effects
fixed = filter(x->contains(x, "X_Jfips"), names(data));
year = filter(x->contains(x, "X_Tyear"), names(data));

In [34]:
census = []
census_var = ["AGE", "BN", "BP", "BZ", "ED", "EL", "HI", "HS", "INC", "LF", "LN", "PI", "PO", "PP", "PV", "SPR", "VS"]

for i in 1:size(census_var,1) 
    append!(census, filter(x->contains(x, census_var[i]), names(data)))
end

In [35]:
################################ Variables ##################################

# Treatment Variable
d = ["logfssl"];

# Outcome Variable
y = ["logghomr"];

# Other Control Variables
X1 = ["logrobr", "logburg", "burg_missing", "robrate_missing"];
X2 = ["newblack", "newfhh", "newmove", "newdens", "newmal"];

In [36]:
#################################  Partial out Fixed Effects ########################

# Variables to be Partialled-out
variable = [y, d,X1, X2, census]
varlis = []

# Partial out Variables in varlist from year and county fixed effect
for i in variable
    append!(varlis,i)
end

In [37]:
# Running the following lines takes aprox. 10 minutes (depends on your CPU)

example = DataFrame(CountyCode = data[:,"CountyCode"]);
rdata = DataFrame(CountyCode = data[:,"CountyCode"]);

for i in 1:size(varlis,1)
    rdata[!,varlis[i]]= residuals(lm(term(Symbol(varlis[i])) ~ sum(term.(Symbol.(year))) + sum(term.(Symbol.(fixed))), data))
end

# DML for neural nets

The following algorithm comsumes $Y$,$D$ and $Z$, and learns the residuals $\tilde{Y}$ and $\tilde{D}$ via a neural network, where the residuals are obtained by cross-validation (cross-fitting). Then, it prints the estimated coefficient β and the clustered standard error from the final OLS regression.

In [14]:
using Flux
using Flux: crossentropy, @epochs
using Flux.Data: DataLoader
using Flux: throttle
using Flux: onehotbatch, onecold, @epochs
using StatsBase

In [41]:
Z

,logrobr,logburg,burg_missing,robrate_missing,newblack,newfhh
,Float64,Float64,Float64,Float64,Float64,Float64
1,0.150893,-0.124395,0.0104613,-0.021229,0.0309471,-0.0204832
2,0.0401683,-0.134781,0.0104613,-0.0194181,0.0309471,-0.0204832
3,-0.017679,-0.167909,0.0104613,-0.0220374,0.0309471,-0.0204832
4,-0.00963344,-0.22925,0.0104613,-0.0194181,0.0309471,-0.0204832
5,-0.0267151,-0.176635,0.00324793,-0.0208037,0.0309471,-0.0204832
6,-0.151487,-0.189069,0.0104613,0.016953,0.0309471,-0.0204832
7,-0.166729,-0.117739,0.0104613,0.0245505,0.0309471,-0.0204832
8,-0.0996453,-0.0833094,0.00448964,0.021457,0.0309471,-0.0204832
9,0.151557,0.319282,-0.0448348,-0.0366629,0.0309471,-0.0204832


In [54]:
mean_1 = mean.(eachcol(z))


std_1 = std.(eachcol(z))



for i in 1:size(z)[2]
    p = (z[:, i] .- mean_1[i]) / std_1[i]
    #colname = names(Z)[i]
    df[!,i] = p
end
    
  

LoadError: ArgumentError: Cannot assign to non-existent column: 1

In [67]:
df[:,1] = p

LoadError: ArgumentError: Cannot assign to non-existent column: 1

In [59]:
p = (z[:,1].-mean_1[1]) / std_1[1]

3900-element Vector{Float64}:
  0.19297177786972372
  0.051369841595354765
 -0.02260902583397552
 -0.012319878683044443
 -0.034165055650964275
 -0.1937313645019607
 -0.21322361685221175
 -0.12743291758859554
  0.19382086474044877
  0.06087839527047379
  0.010413768831547246
 -0.10382424886614387
 -0.19631106003544865
  ⋮
 -0.1519666509001525
 -0.028157516507789738
  0.10057946371092834
  0.07184702189210973
  0.267696062135722
 -0.1097948884357846
 -0.1257385988841877
  0.02330696441580686
  0.3215097989527482
  0.23017669648906897
 -1.7589560388446375
  0.430706036235847

In [45]:
[names(Z) mean_1]

195×3 Matrix{Any}:
 "logrobr"          "logrobr"          -2.84804e-15
 "logburg"          "logburg"           8.66521e-15
 "burg_missing"     "burg_missing"     -1.38787e-17
 "robrate_missing"  "robrate_missing"  -1.22089e-17
 "newblack"         "newblack"          3.02021e-15
 "newfhh"           "newfhh"           -5.07913e-16
 "newmove"          "newmove"           4.74788e-15
 "newdens"          "newdens"          -8.50898e-15
 "newmal"           "newmal"           -1.01486e-17
 "AGE010D"          "AGE010D"          -2.37094e-14
 "AGE050D"          "AGE050D"           8.20774e-15
 "AGE110D"          "AGE110D"           2.68134e-14
 "AGE170D"          "AGE170D"           1.65919e-14
 ⋮                                     
 "PVY020D"          "PVY020D"          -2.85392e-14
 "PVY120D"          "PVY120D"          -3.9999e-14
 "PVY210D"          "PVY210D"          -2.53837e-15
 "PVY310D"          "PVY310D"           5.4832e-14
 "PVY420D"          "PVY420D"          -4.56684e-14
 "PVY52

In [24]:
function DML2_for_NN(z , d , y, nfold, clu, num_epochs)
    
    # Num ob observations
    nobser = size(z,1)
    
    # Define folds indices 
    foldid = collect(Kfold(size(z)[1], nfold))
    
    # Create array to save errors 
    ytil = ones(nobser)
    dtil = ones(nobser)
    println("Folds: " )
    
    # loop to save results
    for i in 1:nfold
        ##############################################
        ################| MODEL D |###################
        model_y= Chain(Dense(size(z,2), 16, relu), 
        Dense(16, 16, relu),
        Dense(16, 1))

        opt = RMSProp()
        loss_y(x, y) = Flux.Losses.mse(model_y(x), y)
        metrics_y(x, y) = Flux.mae(model_y(x), y)
        ps_y = Flux.params(model_y)

        ##############################################
        ################| MODEL Y |###################
        model_d= Chain(Dense(size(z,2), 16, relu), 
        Dense(16, 16, relu),
        Dense(16, 1))

        opt = RMSProp()
        loss_d(x, y) = Flux.Losses.mse(model_d(x), y)
        metrics_d(x, y) = Flux.mae(model_d(x), y)
        ps_d = Flux.params(model_d)

        data_d = DataLoader((z[foldid[i],:]', d[foldid[i]]'))
        data_y = DataLoader((z[foldid[i],:]', y[foldid[i]]'))

    # Lasso regression, excluding folds selected 
    for epoch in 1:num_epochs
        time = @elapsed Flux.train!(loss_y, ps_y, data_y, opt)
    end

    for epoch in 1:num_epochs
        time = @elapsed Flux.train!(loss_d, ps_d, data_d, opt)
    end

    # Predict estimates using the 
    yhat = model_y(z[Not(foldid[i]),:]')';
    ###############################################################################
    dhat = model_d(z[Not(foldid[i]),:]')';
    
        
    # Save errors 
    dtil[Not(foldid[i])] = (d[Not(foldid[i])] - dhat)
    ytil[Not(foldid[i])] = (y[Not(foldid[i])] - yhat)

    println(i)
    end
    
    # Create dataframe 
    data = DataFrame(ytil = ytil, dtil = dtil, clu=clu)
    
    # OLS clustering at the County level
    rfit = reg(data, @formula(ytil ~ dtil +fe(clu)))
    coef_est = coef(rfit)[1]
    se = FixedEffectModels.coeftable(rfit).cols[2]

    println(" coef (se) = ", coef_est ,"(",se,")")
    
    #return rfit, data;
    
end

DML2_for_NN (generic function with 1 method)

# Estimating the effect with DLM for neural nets

In [38]:
# Treatment variable
D = rdata[!,d]

# Outcome variable

Y = rdata[!,y];

# Construct matrix Z
Z = rdata[!, varlis[3:end]];


In [30]:
# Create main variables
z = Matrix(Z);
d = D[!,1];
y = Y[!,1];
clu = rdata[!, :CountyCode];
first(DataFrame(logghomr = y,logfssl = d,CountyCode = clu ),6)

,logghomr,logfssl,CountyCode
,Float64,Float64,Int64
1,-0.134778,0.0961271,1073
2,-0.239622,0.0808094,1073
3,-0.0786772,0.0573399,1073
4,-0.331465,0.0816945,1073
5,-0.31664,0.0253655,1073
6,0.105132,-0.00677726,1073


In [25]:
##
DML2_for_NN(z,d,y,10,clu,100)

Folds: 
1
2
3
4
5
6
7
8
9
10
 coef (se) = 0.08016811689720804([0.059424888983297536])
